## 🐾 OpenAI API 키 발급 및 설정

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# API KEY 정보 확인
import os

print(f"[API KEY]\n{os.environ['OPENAI_API_KEY'][:-15]}" + "*" * 15)

[API KEY]
sk-proj-y4WToRFYqZ-Wj0gO27S4wqQdFi4npu7mKV5RFHUEcIBuE_riXH0aLGMjXEHMztDkz0VQc1I0hsT3BlbkFJq0VCbNlLt9D-MkAmmk_8eyCar687htSINT9IDo93U4--tA7Vzefg0tpbh9s***************


In [4]:
# LangCahin 버전 확인
from importlib.metadata import version

print("[LangChain 관련 패키지 버전]")
for package_name in [
    "langchain",
    "langchain-core",
    "langchain-experimental",
    "langchain-community",
    "langchain-openai",
    "langchain-teddynote",
    "langchain-huggingface",
    "langchain-google-genai",
    "langchain-anthropic",
    "langchain-cohere",
    "langchain-chroma",
    "langchain-elasticsearch",
    "langchain-upstage",
    "langchain-cohere",
    "langchain-milvus",
    "langchain-text-splitters",
]:
    try:
        package_version = version(package_name)
        print(f"{package_name}: {package_version}")
    except ImportError:
        print(f"{package_name}: 설치되지 않음")

[LangChain 관련 패키지 버전]
langchain: 0.3.23
langchain-core: 0.3.51
langchain-experimental: 0.3.4
langchain-community: 0.3.21
langchain-openai: 0.3.12
langchain-teddynote: 0.3.45
langchain-huggingface: 0.1.2
langchain-google-genai: 2.1.2
langchain-anthropic: 0.3.10
langchain-cohere: 0.4.3
langchain-chroma: 0.2.2
langchain-elasticsearch: 0.3.2
langchain-upstage: 설치되지 않음
langchain-cohere: 0.4.3
langchain-milvus: 0.1.9
langchain-text-splitters: 0.3.8


In [ ]:
# # LangSmith 추적 설정 https://smith.langchain.com
# # .env 파일에 LANGCHAIN_API_KEY 입력
# # !pip install -qU langchain-teddynote
# from langchain_teddynote import logging

# # 프로젝트 이름을 입력
# logging.langsmith("***")

LangSmith 추적을 시작합니다.
[프로젝트명]
PromptIkkyunFinal


## 🐻 데이터셋 1000개 최종ver
부정 리뷰 500개 (1000 rows)

In [ ]:
# 라이브러리 로드
import pandas as pd
import json
from dotenv import load_dotenv
from langchain.prompts import load_prompt
from langchain.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

# .env 로드
load_dotenv()


# 변경된 Pydantic 출력 모델 정의
class SarcasmPair(BaseModel):
    context: str = Field(description="리뷰 내용을 요약한 문맥")
    sarcastic_version: str = Field(description="풍자적으로 재작성된 문장")
    non_sarcastic_version: str = Field(description="풍자 없이 사실만을 전달한 문장")
    sarcastic_explanation: str = Field(
        description="풍자 문장이 왜 풍자인지에 대한 설명"
    )
    non_sarcastic_explanation: str = Field(
        description="비풍자 문장이 왜 풍자가 아닌지에 대한 설명"
    )


# Output parser
parser = PydanticOutputParser(pydantic_object=SarcasmPair)

# 프롬프트 로딩
prompt = load_prompt("sarcasm_prompt.yaml", encoding="utf-8")

# 모델 설정
llm_4o = ChatOpenAI(temperature=0.5, model_name="gpt-4o")
llm_mini = ChatOpenAI(temperature=0.5, model_name="gpt-4o-mini")

chain_4o = prompt | llm_4o | parser
chain_mini = prompt | llm_mini | parser

# 리뷰 500개 로드
df = pd.read_csv("negative_reviews.csv")
sample_df = df.sample(n=500, random_state=777).reset_index(drop=True)

# 실행
results = []
for idx, text in enumerate(sample_df["text"]):
    model_chain = chain_4o if len(text) <= 50 else chain_mini
    try:
        result = model_chain.invoke({"original": text})

        # 풍자 버전
        results.append(
            {
                "context": result.context,
                "response": result.sarcastic_version,
                "label": "Sarcasm",
                "explanation": result.sarcastic_explanation,
            }
        )

        # 비풍자 버전
        results.append(
            {
                "context": result.context,
                "response": result.non_sarcastic_version,
                "label": "Non-Sarcasm",
                "explanation": result.non_sarcastic_explanation,
            }
        )

        if idx % 100 == 0:
            print(f"진행 상황: {idx} / 500 문장 처리 완료")

    except Exception as e:
        print(f"[⚠️] 오류 발생 - {text[:30]}... → {e}")
        continue

# 저장
jsonl_path = "review_train.jsonl"
csv_path = "review_train.csv"

with open(jsonl_path, "w", encoding="utf-8") as f:
    for item in results:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

pd.DataFrame(results).to_csv(csv_path, index=False, encoding="utf-8-sig")

print(f"저장 완료: {jsonl_path} / {csv_path}")
print(f"총 샘플 수: {len(results)}개")

No `_type` key found, defaulting to `prompt`.


진행 상황: 0 / 500 문장 처리 완료
진행 상황: 100 / 500 문장 처리 완료
진행 상황: 200 / 500 문장 처리 완료
진행 상황: 300 / 500 문장 처리 완료
진행 상황: 400 / 500 문장 처리 완료
저장 완료: Ikkyun_1000.jsonl / Ikkyun_1000.csv
총 샘플 수: 1000개


### 결과요약

토큰 사용량: 373,209  
비용: $1.05  
런타임: 28분  
비고: 품질 괜찮음.

## 🐼 테스트셋 200개
트레이닝셋과 겹치지 않는 부정 리뷰 100개 (200 rows)

In [ ]:
# 라이브러리 로드
import pandas as pd
import json
from dotenv import load_dotenv
from langchain.prompts import load_prompt
from langchain.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

# .env 로드
load_dotenv()


# 출력 모델 정의
class SarcasmPair(BaseModel):
    context: str = Field(description="리뷰 내용을 요약한 문맥")
    sarcastic_version: str = Field(description="풍자적으로 재작성된 문장")
    non_sarcastic_version: str = Field(description="풍자 없이 사실만을 전달한 문장")
    sarcastic_explanation: str = Field(
        description="풍자 문장이 왜 풍자인지에 대한 설명"
    )
    non_sarcastic_explanation: str = Field(
        description="비풍자 문장이 왜 풍자가 아닌지에 대한 설명"
    )


# 파서 설정
parser = PydanticOutputParser(pydantic_object=SarcasmPair)

# 프롬프트 및 체인 설정
prompt = load_prompt("sarcasm_prompt.yaml", encoding="utf-8")
llm_4o = ChatOpenAI(temperature=0.5, model_name="gpt-4o")
llm_mini = ChatOpenAI(temperature=0.5, model_name="gpt-4o-mini")
chain_4o = prompt | llm_4o | parser
chain_mini = prompt | llm_mini | parser

# 전체 리뷰 로드
df_all = pd.read_csv("negative_reviews.csv")

# 기존에 사용된 500개 리뷰 로드r
used_df = pd.read_csv("review_train.csv")
used_texts = set(used_df["context"].unique())  # context 기준으로 중복 제거

# 사용되지 않은 리뷰 필터링
unused_df = df_all[~df_all["text"].isin(used_texts)].reset_index(drop=True)

# 새로 샘플링할 100개 추출
sample_df = unused_df.sample(n=100, random_state=123).reset_index(drop=True)

# 생성 실행
results = []
for idx, text in enumerate(sample_df["text"]):
    model_chain = chain_4o if len(text) <= 50 else chain_mini
    try:
        result = model_chain.invoke({"original": text})

        results.append(
            {
                "context": result.context,
                "response": result.sarcastic_version,
                "label": "Sarcasm",
                "explanation": result.sarcastic_explanation,
            }
        )
        results.append(
            {
                "context": result.context,
                "response": result.non_sarcastic_version,
                "label": "Non-Sarcasm",
                "explanation": result.non_sarcastic_explanation,
            }
        )

        if idx % 20 == 0:
            print(f"[테스트셋 진행 상황] {idx} / 100 리뷰 처리 완료")

    except Exception as e:
        print(f"[⚠️] 오류 - {text[:30]}... → {e}")
        continue

# 저장
jsonl_path = "review_test.jsonl"
csv_path = "review_test.csv"

with open(jsonl_path, "w", encoding="utf-8") as f:
    for item in results:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

pd.DataFrame(results).to_csv(csv_path, index=False, encoding="utf-8-sig")

print(f"테스트셋 저장 완료: {jsonl_path} / {csv_path}")
print(f"총 샘플 수: {len(results)}개")

No `_type` key found, defaulting to `prompt`.


[테스트셋 진행 상황] 0 / 100 리뷰 처리 완료
[테스트셋 진행 상황] 20 / 100 리뷰 처리 완료
[테스트셋 진행 상황] 40 / 100 리뷰 처리 완료
[테스트셋 진행 상황] 60 / 100 리뷰 처리 완료
[테스트셋 진행 상황] 80 / 100 리뷰 처리 완료
테스트셋 저장 완료: Ikkyun_test200.jsonl / Ikkyun_test200.csv
총 샘플 수: 200개


### 결과요약

토큰 사용량: 74,968  
비용: $0.22  
런타임: 7분  

## 🐻‍❄️ 추가 데이터 250개
겹치지 않는 부정 리뷰 125개 (250 rows)

In [6]:
# 라이브러리 로드
import pandas as pd
import json
from dotenv import load_dotenv
from langchain.prompts import load_prompt
from langchain.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

# .env 로드
load_dotenv()


# 출력 모델 정의
class SarcasmPair(BaseModel):
    context: str = Field(description="리뷰 내용을 요약한 문맥")
    sarcastic_version: str = Field(description="풍자적으로 재작성된 문장")
    non_sarcastic_version: str = Field(description="풍자 없이 사실만을 전달한 문장")
    sarcastic_explanation: str = Field(
        description="풍자 문장이 왜 풍자인지에 대한 설명"
    )
    non_sarcastic_explanation: str = Field(
        description="비풍자 문장이 왜 풍자가 아닌지에 대한 설명"
    )


# 파서 설정
parser = PydanticOutputParser(pydantic_object=SarcasmPair)

# 프롬프트 및 체인 설정
prompt = load_prompt("sarcasm_prompt.yaml", encoding="utf-8")
llm_4o = ChatOpenAI(temperature=0.5, model_name="gpt-4o")
llm_mini = ChatOpenAI(temperature=0.5, model_name="gpt-4o-mini")
chain_4o = prompt | llm_4o | parser
chain_mini = prompt | llm_mini | parser

# 전체 리뷰 로드
df_all = pd.read_csv("negative_reviews.csv")

# 기존 사용된 context 로드 (train + test)
train_df = pd.read_csv("review_train.csv")
test_df = pd.read_csv("review_test.csv")
used_contexts = set(train_df["context"].unique()).union(
    set(test_df["context"].unique())
)

# 사용되지 않은 리뷰 필터링
unused_df = df_all[~df_all["text"].isin(used_contexts)].reset_index(drop=True)

# 125개 샘플링
sample_df = unused_df.sample(n=125, random_state=456).reset_index(drop=True)

# 생성 시작
results = []
for idx, text in enumerate(sample_df["text"]):
    model_chain = chain_4o if len(text) <= 50 else chain_mini
    try:
        result = model_chain.invoke({"original": text})

        results.append(
            {
                "context": result.context,
                "response": result.sarcastic_version,
                "label": "Sarcasm",
                "explanation": result.sarcastic_explanation,
            }
        )
        results.append(
            {
                "context": result.context,
                "response": result.non_sarcastic_version,
                "label": "Non-Sarcasm",
                "explanation": result.non_sarcastic_explanation,
            }
        )

        if idx % 25 == 0:
            print(f"[진행 상황] {idx} / 125 리뷰 처리 완료")

    except Exception as e:
        print(f"[⚠️] 오류 - {text[:30]}... → {e}")
        continue

# 저장
jsonl_path = "Ikkyun_new.jsonl"
csv_path = "Ikkyun_new.csv"

with open(jsonl_path, "w", encoding="utf-8") as f:
    for item in results:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

pd.DataFrame(results).to_csv(csv_path, index=False, encoding="utf-8-sig")

print(f"저장 완료: {jsonl_path} / {csv_path}")
print(f"총 생성 샘플 수: {len(results)}개")

No `_type` key found, defaulting to `prompt`.


[진행 상황] 0 / 125 리뷰 처리 완료
[진행 상황] 25 / 125 리뷰 처리 완료
[진행 상황] 50 / 125 리뷰 처리 완료
[진행 상황] 75 / 125 리뷰 처리 완료
[진행 상황] 100 / 125 리뷰 처리 완료
저장 완료: Ikkyun_new.jsonl / Ikkyun_new.csv
총 생성 샘플 수: 250개


### 결과요약

토큰 사용량: 93,899  
비용: $0.29  
런타임: 7분  